Again, let's start with loading the most important modules and setting some options.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None
%matplotlib inline
sns.set_style("ticks")

# Geo Basics

### Geodata

Geodata is information about geographic objects (**features**) and their precise (geo)spatial location. The geometric shape and associated attributes of geographic objects are modeled together either in a **vector data model** or **raster data model**.

**Raster**: 
- Built up of regularly arranged and equally sized pixels (**grid**).
- The (color) values of the individual pixels describe the geographical area associated with the pixel.
- The area covered by a single pixel determines the spatial resolution of the grid (e.g. 1x1km).
- Are mainly used for continuous geographic phenomena - especially for physical phenomena (e.g. elevation).

**Vector**: 
- Geographic features are modeled as **points**, **lines**, and **polygons**.
- Points can be precisely defined by a pair of **coordinates** (x, y; latitude, longitude).
- Lines and polygons consist of interconnected (ordered) points.
- Geometries of geographic features can be defined exactly in this way.
- Used mainly for discrete geographic phenomena - e.g. administrative boundaries, buildings, etc.

<img src="../misc/geodata_models.png" width="600"> 

### Layer concept

In **Geographic Information Systems** (**GIS**), geodata is structured using the layer principle. Each layer represents a distinct geographic phenomenon and several layers together represent a more or less abstracted version of the real world. 

<img src="../misc/layer.jpg" width="300"> 

### Coordinate (reference) systems

- System used to indicate the position of a point in space.
- Serves for the localization of geographical objects.
- The position is indicated by coordinates (X,Y).
- Approximately 4,400 different coordinate reference systems (**CRS**).
- **EPSG** system with unique codes for each CRS.
- Coordinate transformation (conversion) between different coordinate systems is possible.
- **Geographic (spherical) coordinates** vs. **projected (2D) coordinates**.
- Representation and analysis is done in the projected CRS.

<img src="../misc/crs.png" width="250"> 

**Geographic coordinate systems**:
- Spherical coordinates.
- Longitude and latitude.
- Specified in degree or decimal format.
- Coordinates are given by default as geographic coordinates in decimal format (e.g. ISTARI.AI HQ: 49.47357504318401, 8.475272386708715).
- Current standard is the worldwide valid **WGS 84** system (**EPSG: 4326**).

<img src="../misc/globe.gif" width="350"> 

**Projected coordinate systems**:
- Curved surface of the earth is projected onto a plane
- The mathematical operation of projection leads to distortions in Distance, Area, Shape, and/or Direction.
- Many projections work only in certain regions.
- **EPSG: 3857 "Web Mercator"** standard for Web Map Services (WMS) like Google Maps and OpenStreetMap.

![SegmentLocal](../misc/animation_projection.gif)

# 4. GeoPandas

We will use GeoPandas to create maps. GeoPandas extends the dataypes used by Pandas to allow for spatial operations on geometric types. Let's install GeoPandas. This can be a sometimes a littel bit tricky. I recommend to first install fiona (a dependency of GeoPandas):

1. ```conda install -c conda-forge fiona```

then try to install geopandas:

2. ```pip install geopandas```

If this does not work, try to run the following commands:
- ```pip install wheel```
- ```pip install pipwin```
- ```pipwin install shapely```
- ```pipwin install gdal```
- ```pipwin install fiona```
- ```pipwin install pyproj```
- ```pipwin install six```
- ```pipwin install rtree```
- ```pipwin install geopandas```
- ```pipwin install descartes```
- ```pip install mapclassify```

After restarting the kernel (use the circle arrow above) we should be able to import geopandas:

In [ ]:
import geopandas as gpd

Now we can use geopandas to read geodata (files that contain tabular data and geometry). The most common geodata filetypes are:
- **ESRI shapefile**: ESRI is "the Microsoft of Geographic Information Systems" and shapefiles are their standard file format. A shapefile consists of several files (3 to 5) but you want to work with the .shp file which contains the geometry.
- **Google KML/KMZ**: .kml and .kmz files are used by Google Maps and Google Earth.
- **GeoJSON**: A open data format for geodata based on the popular JSON format. 

Let's read in a GeoJSON file I prepared for us. You can see that it looks very much like a normal dataframe after loading.

In [ ]:
gpd.read_file("../data/world_map.json")

Let's have a look at the datatypes of this dataframe. The interesting one is the last one. It is named "geometry" and has our new dataframe datatype ```geometry```.

In [ ]:
world_map = gpd.read_file("../data/world_map.json")
world_map.dtypes

Let's have a look at the "geometry" of a single observation. As you can see, it includes a ```POLYGON``` made up of coordinate pairs (longitude and latitude), which trace the border of the polygon. 

In [ ]:
world_map.sample()["geometry"]

The geometry stored in this column allows us to plot the dataframe as a map.

In [ ]:
world_map.plot(figsize=(20,20))

If we want to plot a single observation from our geopandas dataframe (corresponding to a single country), we can simply filter that observation and plot it.

In [ ]:
world_map[world_map["sovereignt"] == "Germany"].plot()

We can also select several observations and plot them.

In [ ]:
world_map[world_map["sovereignt"].isin(["Germany", "Austria"])].plot()

### Coordinate Reference Systems (CRS) and projections

We already learnt about CRS and that all geodata needs to have an associated CRS in order to be located on the earth's surface and displayed properly. We can easily check for a geopanda dataframe's associated CRS by using ```.crs```. 

In [ ]:
world_map.crs

It makes sense that ```world_map```'s CRS is WGS 84 (EPSG code: 4326) given that it is geodata with global coverage. Maybe you already noticed that Germany looks somehow distorted and unfamiliar. This is because you are used to see maps of Germany in projected CRS that work well for central Europe. For Germany, UTM zone 32 (https://epsg.io/25832) is usually used (EPSG: 25832) and we can easily transfer our selection of ```world_map``` to this crs using ```to_crs(epsg="25832")```.

In [ ]:
world_map[world_map["sovereignt"] == "Germany"].to_crs(epsg="25832").plot()

Note, however, that this CRS would not work globally.

In [ ]:
world_map.to_crs(epsg="25832").plot(figsize=(10,10))

### Choropleth maps

We can also pass a column to the ```plot()``` function in order to use it for the coloring of the countries (creating a so-called choropleth map https://en.wikipedia.org/wiki/Choropleth_map).

In [ ]:
world_map.plot(figsize=(20,20), column="pop_est")

The classes used to color the countries are not that informative yet, as most countries are in the lowest (i.e. darkest) class. We can change that by selecting a more appropriate choropleth classification scheme. Here are some commonly used ones:
- ```'NaturalBreaks'``` https://en.wikipedia.org/wiki/Jenks_natural_breaks_optimization
- ```'Quantiles'``` https://en.wikipedia.org/wiki/Quantile
- ```'Percentiles'``` https://en.wikipedia.org/wiki/Percentile
- ```'StdMean'``` https://en.wikipedia.org/wiki/Standard_deviation

We may also pass a ```k=``` parameter to define the number of classes we want to create using the defined scheme. We also want a legend for our map which we can achieve by passing ```legend=True```.

In [ ]:
world_map.plot(figsize=(20,20), column="pop_est", scheme="Quantiles", k=10, legend=True)

In the legend, we can see that the lowest value for population is "-99", which makes no sense. It seems like that the person who created this dataset used "-99" to mark missing values. Let's replace them.

In [ ]:
world_map["pop_est"] = world_map["pop_est"].replace({-99: np.nan})

Let's also create a new column containing the population in millions.

In [ ]:
world_map["pop_million"] = world_map["pop_est"] / 1000000

We can also pass some ```missing_kwds={}``` to tell geopandas how to handle missing data.

In [ ]:
world_map.plot(figsize=(20,20), column="pop_million", scheme="NaturalBreaks", k=5, legend=True, missing_kwds={'color': 'lightgrey', "label": "Missing"})

To make some proper maps, we usually want to use the matplotlib approach we learned about last week.

In [ ]:
from matplotlib import gridspec

# create one figure size 20,10
fig = plt.figure(figsize=(20,10))
# use the gridspec function to define 2 columns in this figure; set the width ratio to 3:1, meaning the first column will be 3 times the width of the second column
columns = gridspec.GridSpec(ncols=2, nrows=1, width_ratios=[3, 1])

# add a subplot to the first column of the figure
axis_0 = fig.add_subplot(columns[0])
# world population choropleth map
world_map.plot(column="pop_million", scheme="NaturalBreaks", k=5, legend=True,
               legend_kwds={"loc": "lower left"},
               missing_kwds={'color': 'lightgrey', "label": "Missing"},
               ax=axis_0)
axis_0.set_axis_off()


# add a subplot to the second column of the figure
axis_1 = fig.add_subplot(columns[1])
# world population histogram
sns.distplot(world_map["pop_million"], kde=False, bins=50, ax=axis_1)
axis_1.set_xlabel("Population in Millions")
axis_1.set_ylabel("Frequency")
axis_1.set_xlim(0,1500)


# Final adjustments
fig.tight_layout() #tidy up the figure 
plt.savefig("world_population.png", dpi=150) #save the figure

Remember the nice color palettes from seaborn? --> https://seaborn.pydata.org/tutorial/color_palettes.html

In [ ]:
sns.palplot(sns.color_palette("Paired"))

We can also get the color hex codes for these colors using the ```as_hex()``` method on the palette.

In [ ]:
sns.color_palette("Paired").as_hex()

If we want to use one these in geopandas, we have to transfer the seaborn color palette hex codes to a matplotlib (listed) colormap.

In [ ]:
from matplotlib.colors import ListedColormap

sns_color_palette = ListedColormap(sns.color_palette("Paired").as_hex())

Let's update the map we have created above with some more beautiful seaborn colors. For the histogram, we can select the second color of the palette directly.

In [ ]:
# create one figure size 20,10
fig = plt.figure(figsize=(20,10))
# use the gridspec function to define 2 columns in this figure; set the width ratio to 3:1, meaning the first column will be 3 times the width of the second column
columns = gridspec.GridSpec(ncols=2, nrows=1, width_ratios=[3, 1])

# add a subplot to the first column of the figure
axis_0 = fig.add_subplot(columns[0])
# world population choropleth map
world_map.plot(column="pop_million", scheme="NaturalBreaks", k=5, legend=True, cmap=sns_color_palette,
               legend_kwds={"loc": "lower left", },
               missing_kwds={'color': 'lightgrey', "label": "Missing"},
               ax=axis_0)
axis_0.set_axis_off()


# add a subplot to the second column of the figure
axis_1 = fig.add_subplot(columns[1])
# world population histogram
sns.distplot(world_map["pop_million"], kde=False, bins=50, ax=axis_1, color=sns.color_palette("Paired")[4])
axis_1.set_xlabel("Population in Millions")
axis_1.set_ylabel("Frequency")
axis_1.set_xlim(0,1500)


# Final adjustments
fig.tight_layout() #tidy up the figure 
plt.savefig("world_population_updated.png", dpi=150) #save the figure

### Combining geodata and non-geographic data

Let's combine our world map with some university ranking data. First we load the university ranking data we have been working on over the past two weeks.

In [ ]:
ranking = pd.read_csv("../data/times_university_data.csv", sep=",")

Remember that the "world_rank" column was "object" type because there are rankings like "250-300" included. Let's transfer that column to numeric and drop the observations which we cannot transfer.

In [ ]:
ranking["world_rank"]

In [ ]:
ranking["world_rank"] = pd.to_numeric(ranking["world_rank"], errors="coerce")

Let's select only the world's top 100 universities in 2016.

In [ ]:
top100 = ranking[(ranking["year"] == 2016) & (ranking["world_rank"] <= 100)]

About a third of the top 100 universities are from the USA.

In [ ]:
country_counts = top100["country"].value_counts()
country_counts

Let's use pandas ```groupby(by=[])``` method to group our dataframe by countries and then calculate the mean "citations" of each country's universities in the top 100.

In [ ]:
mean_top100_citations = top100.groupby(by=["country"])["citations"].mean()
mean_top100_citations

In [ ]:
world_map["sovereignt"]

Now we can merge our ```world_map``` geopandas dataframe with the two Pandas series (remember, those are like single column dataframes).

In [ ]:
world_map = world_map.merge(country_counts, how="left", left_on="sovereignt", right_index=True).rename(columns={"country": "n_universities"})
world_map = world_map.merge(mean_top100_citations, how="left", left_on="sovereignt", right_index=True)

And now we can use these two new columns for some fresh new maps.

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(10,10))

# number of top 100 universities map
world_map.plot(figsize=(20,20), column="n_universities", scheme="UserDefined", classification_kwds={'bins':[1, 10, 20, 30]}, legend=True, missing_kwds={'color': 'lightgrey', "label": "No top 100 universities"}, ax=axes[0])
axes[0].set_axis_off()
axes[0].set_title("Number of Universities in Top 100",fontdict={"fontsize":20})

# mean citations map
world_map.plot(figsize=(20,20), column="citations", scheme="NaturalBreaks", k=5, legend=True, missing_kwds={'color': 'lightgrey', "label": "No top 100 universities"}, ax=axes[1])
axes[1].set_axis_off()
axes[1].set_title("Mean Citations of Top 100 Universities",fontdict={"fontsize":20})

# Final adjustments
fig.tight_layout() #tidy up the figure 
plt.savefig("dual_map.png", dpi=150) #save the figure

The number of top 100 universities by population may be something we are interested in.

In [ ]:
world_map["top_ranking_per_1m_pop"] = world_map["n_universities"] / world_map["pop_million"]

Looking at the descriptive statistics there seems something wrong. The maximum is very large.

In [ ]:
world_map["top_ranking_per_1m_pop"].describe()

When we look at our observations that have a non-null value in the new "top_ranking_per_1m_pop" column, we see that UK, USA, and France appear more than once and have some suspiciously high "top_ranking_per_1m_pop" values.

In [ ]:
world_map[world_map["sovereignt"] == "United Kingdom"]

It turns out that, for the example of UK, the Falkland Islands are a separat "Dependency" in the "type" column.

In [ ]:
world_map[world_map["sovereignt"] == "United Kingdom"]

There are actually several country types that sometimes share name like "United Kingdom".

In [ ]:
world_map["type"].value_counts()

So let's recalculate the "top_ranking_per_1m_pop" column in a separat geopandas dataframe which includes "country" and "Sovereign country" types only (no "Dependency" and the like)

In [ ]:
del(world_map["top_ranking_per_1m_pop"]) # delete wrong column in original dataframe

world_map_selection = world_map[world_map["type"].isin(["Sovereign country", "Country"])]
world_map_selection["top_ranking_per_1m_pop"] = world_map_selection["n_universities"] / world_map_selection["pop_million"]

In [ ]:
world_map_selection["top_ranking_per_1m_pop"].describe()

Now it makes more sense! 

We can actually see that smaller countries like the Netherlands and Switzerland outperform the USA for example. Both have about three times the top 100 universities per million inhabitants than the US.

In [ ]:
world_map_selection[world_map_selection["top_ranking_per_1m_pop"].notnull()][["sovereignt", "top_ranking_per_1m_pop"]].sort_values(by=["top_ranking_per_1m_pop"], ascending=False)

In [ ]:
world_map_selection.plot(figsize=(30,30), column="top_ranking_per_1m_pop", scheme="NaturalBreaks", k=5, legend=True, missing_kwds={'color': 'lightgrey', "label": "No top 100 universities"})

### Spatial joins

The **spatial join** joins (merges) attributes from one feature to another based on the spatial relationship. It is one of the most commonly used tools in GIS and can be thought of as the geographic equivalent to Panda's ```.merge()``` function. Other than with ```.merge()```, geopanda's ```.sjoin()``` uses the location of geographic features instead of keys to combine observations from two dataframes.

There are three types of spatial joins available:
- ```.sjoin(how="left")```: The default. Keep all observations from "left" dataframe and join (if possible) information from right dataframe. Duplicate left observations if more than one match with right observation occurs.
- ```.sjoin(how="right")```: Just the other way around.
- ```.sjoin(how="inner")```: Keep all observations from both "left" and "right" dataframe. Also duplicate observations if necessary.

Additionally, you can control the type of intersection required between two features to be matched:
- ```.sjoin(how="left", op="intersects")```: The default. Matches two features if their geometries overlap.
- ```.sjoin(how="left", op="contains")```: Matches two features if B is completly wihin A (no "sticking out").
- ```.sjoin(how="left", op="within")```: Like "contains", but the other way around.

<img src="../misc/spatial_join.gif" width="400"> 

For demonstration purposes, let's create a point dataset first which covers the entire bounding box (```.total_bounds```) of our world_map geometries.

In [ ]:
world_bounding_box = [int(x) for x in world_map.total_bounds]
print(world_bounding_box)

We can now use this bounding box to create a geopandas ```GeoDataFrame()``` consisting of a number of points along the diagonal.

In [ ]:
from shapely.geometry import Point

number_of_points = 10
diagonal_points = gpd.GeoDataFrame([
    {'geometry': Point(x, y)}
    for x, y in zip(range(world_bounding_box[0], world_bounding_box[2], int((world_bounding_box[2] - world_bounding_box[0]) / number_of_points)),
                    range(world_bounding_box[1], world_bounding_box[3], int((world_bounding_box[3] - world_bounding_box[1]) / number_of_points)))])

diagonal_points.crs = {"init":"epsg:4326"}
#diagonal_points = diagonal_points.set_crs(epsg="4326")
diagonal_points.plot(ax=world_map.plot(figsize=(10,10)), color="black")

If we now want to know which point lies within which country, we can match both geometries using geopanda's ```sjoin()```.

In [ ]:
gpd.sjoin(diagonal_points, world_map[["sovereignt", "geometry"]], how="left")

### Calculating distances

Calculating distances between features can be done with geopandas using ```.distance()``` on a feature's geometry and passing another feature's geometry.

Let's first store our spatial join output to a variable and then select two points from it. In order to get back our distance calculation in meters rather than degrees, we need to set the crs of both features to a meter-based global crs like "web mercator" (EPSG: 3857).

In [ ]:
diagonal_points_sjoined = gpd.sjoin(diagonal_points, world_map[["sovereignt", "geometry"]], how="left")

point_in_russia = diagonal_points_sjoined[diagonal_points_sjoined["sovereignt"] == "Russia"].to_crs(epsg="3857")
point_in_ghana = diagonal_points_sjoined[diagonal_points_sjoined["sovereignt"] == "Ghana"].to_crs(epsg="3857")

In [ ]:
point_in_ghana

Using both features' ```geometry``` column and extracting their ```.values``` allows us to calculate the ```.distance``` between them.

In [ ]:
point_in_russia["geometry"].values.distance(point_in_ghana["geometry"].values)

If we want to know the distance in km, we just need to divide by 1000.

In [ ]:
point_in_russia["geometry"].values.distance(point_in_ghana["geometry"].values) / 1000

By the way, we can also use these two points we have selected to create a ```LineString``` geometry. Thereby, one point constitutes the start and the other one the end of the LineString which we us as the ```geometry``` of a new geodataframe.

In [ ]:
from shapely.geometry import LineString

x1 = point_in_russia["geometry"].to_crs(epsg="4326").values.x
y1 = point_in_russia["geometry"].to_crs(epsg="4326").values.y

x2 = point_in_ghana["geometry"].to_crs(epsg="4326").values.x
y2 = point_in_ghana["geometry"].to_crs(epsg="4326").values.y

line_between_ghana_and_russia = gpd.GeoDataFrame([{'geometry': LineString([[x1, y1], [x2, y2]])}])
line_between_ghana_and_russia.crs = {"init":"epsg:4326"}
#line_between_ghana_and_russia = line_between_ghana_and_russia.set_crs(epsg="4326")

In [ ]:
line_between_ghana_and_russia.plot(ax=world_map.plot(figsize=(10,10)), color="black")

# 5. Geocoding

The last thing we want to do today is to *geocode* the locations of all universities in Germany that are included in the ranking. 

Geocoding describes the process where you transform the textual description of a location (e.g. a postal address) to a geographic object with a defined position on the surface of the Earth. Most of you do it on a daily basis when you use the search function in Google Maps or Apple Maps on your smartphone.

Let's first extract the German universities and put them into a separat dataframe.

In [ ]:
german_universities = ranking[(ranking["country"] == "Germany") & (ranking["year"] == 2016)]
german_universities.head(5)

There is not much location information included. But we can try to geocode the universties using their name (which often includes the city) and the "country" column. Let's create a "search_string" columns which contains both.

In [ ]:
german_universities["search_string"] = german_universities["university_name"] + " " + german_universities["country"]
german_universities.head(5)

For the actual geocoding, we use a Python package called Geocoder. Let's install it using pip:

```pip install geocoder```

Let's also install contextily which will allow us to plot "basemaps" (background maps). Also use pip for that:

```pip install contextily```

Geocoding with the Geocoder package is super easy. There are several geocoding services you may use. Some of them will be familiar to you. Some require you to create an account before you can use them (you have to pass your login information together with the geocoding request then) and all of them will prevent you from geocoding thousands of observations (for that you usually have to pay).

For now we will use the ESRI arcgis geocoder. Check out the different geocoding providers here: https://geocoder.readthedocs.io/

We just have to pass a search string to ```geocoder.arcigs()```.

In [ ]:
import geocoder

geocoding_results = geocoder.arcgis('Darmstadt, Germany')

When we have a look at the results, we can see that the geocoding worked (```[OK]```) and "Darmstadt, Hessen" has been found as the correct location. 

In [ ]:
geocoding_results

Actually Geocoder will return us a json file with a lot of information 

In [ ]:
geocoding_results.json

Compare that to the results of a query for a more specific location.

In [ ]:
geocoder.arcgis('Hochschule Darmstadt D-19, Darmstadt, Deutschland').json

The information we are actually interested in are **longitute** and **latitude** (```lon``` and ```lat```) which define the postion of our result on the surface of the earth --> 
https://en.wikipedia.org/wiki/Geographic_coordinate_system

We can get that information as a list directly from the geocoding results using the ```latlng``` method.

In [ ]:
geocoding_results.latlng

So let's create a new column in our German universities dataframe with the lon and lat of the universities by using the ```apply(lambda x:)``` method and passing the "search_string" we have created for each university.

In [ ]:
german_universities["lonlat"] = german_universities["search_string"].apply(lambda x: geocoder.arcgis(x).latlng)

Let's have a look.

In [ ]:
german_universities.head(1)

Okay, seems like all our universities now have a filled "lonlat" column.

In [ ]:
german_universities[german_universities["lonlat"].isnull()]

Let's split the longitute and latitude information into two separat columns.

In [ ]:
german_universities["lat"] = german_universities["lonlat"].apply(lambda x: x[0])
german_universities["lon"] = german_universities["lonlat"].apply(lambda x: x[1])

In [ ]:
german_universities.head(2)

Now we can use geopandas to transfer our pandas dataframe into a geodataframe by telling it to generate the ```geometry=points_from_xy()``` from the columns that store the longitute and latitute values.

In [ ]:
german_universities = gpd.GeoDataFrame(german_universities, geometry=gpd.points_from_xy(german_universities["lon"], german_universities["lat"]))

In [ ]:
german_universities.head(2)

In [ ]:
german_universities.plot()

Now we can plot both geodataframes within the same subplot by passing the subplot's axis ```ax=ax``` and by defining the order of the two layers using ```zorder=```. However we already saw above that the shape of Germany is not very nice because we are using a low resolution map which is inteneded for global maps. So first we load a high resolution map of europe...

In [ ]:
europe_map = gpd.read_file("../data/europe_map.json")

...and select Germany.

In [ ]:
germany = europe_map[europe_map["sovereignt"] == "Germany"]
germany.plot()

In [ ]:
germany.boundary.plot()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

germany.boundary.plot(ax=ax, zorder=1, color="black") # using .boundary tells geopandas to only plot the boundary of polygons
points = german_universities.plot(ax=ax, zorder=2, column="world_rank", scheme="NaturalBreaks", legend=True, missing_kwds={'color': 'white', "label": "Ranking worse than 200"})
ax.set_axis_off()
ax.set_title("Ranked Universities in Germany", fontdict={"fontsize":20})

# Final adjustments
fig.tight_layout() #tidy up the figure 
plt.savefig("ranked_german_universities.png", dpi=150) #save the figure

Let's also load contextily for a nice basemap and define a basemap provider.

In [ ]:
import contextily as cx

basemap_provider = cx.providers.CartoDB.Voyager

Before we can add a basemap to our plot, we have to change the coordinate reference system ```crs``` of our geodata. You can read about that on wikipedia https://en.wikipedia.org/wiki/Geographic_coordinate_system or just think about it as translating both datasets in the same format. 

In [ ]:
germany = germany.set_crs("EPSG:4326")
germany = germany.to_crs(epsg=3857)

german_universities = german_universities.set_crs("EPSG:4326")
german_universities = german_universities.to_crs(epsg=3857)

Now we can add a basemap using contextily ```add_basemap()```.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 20))

#germany.boundary.plot(ax=ax, zorder=1, color="black")
german_universities.plot(ax=ax, zorder=2, column="world_rank", scheme="NaturalBreaks", legend=True, missing_kwds={'color': 'grey', "label": "Ranking worse than 200"})
cx.add_basemap(ax, zoom="auto", source=basemap_provider)
ax.set_axis_off()
ax.set_title("Ranked Universities in Germany", fontdict={"fontsize":20})

# Final adjustments
fig.tight_layout() #tidy up the figure 
plt.savefig("ranked_german_universities_basemap.png", dpi=150) #save the figure

# 6. Spatial statistics

Let's take a *very* quick look at **spatial statistics**. Spatial statistics is used to statistically explore and assess geographic patterns. The underlying concept here is **spatial dependence** which is the spatial relationship of spatial variables and geographic features which are treated as random variables. It measures the existence of **statistical dependence** between those random variables. So it more or less boils down to whether observed **geogprahic patterns are random or not**. We can determine this by measuring for **spatial autocorrelation**. Measures of spatial autocorrelation describe the degree to which observations (values) at different spatial locations, are similar to each other and whether this is by chance (i.e. randomness) or not. 

Spatial autocorrelation can be positive (**clustering**) or negative (**dispersion/uniformity/regularity**) or there can be **complete spatial randomness** (**CSR**).

<img src="../misc/spatial_autocorrelation.png" width="600"> 
<img src="../misc/spatial_autocorrelation2.jpg" width="600"> 

Spatial autocorrelation in a variable can be exogenous (it is caused by another spatially autocorrelated variable, e.g. landslides due to rainfall) or endogenous (it is caused by the process at play, e.g. the spread of a disease). Present spatial autocorrelation can **violate the assumptions of standard statistical techniques** that assume independence among observations. For example, regression analyses that do not compensate for spatial dependency can have unstable parameter estimates and yield unreliable significance tests.

### Moran's I

The more or less standard statistical test for (global) spatial autocorrelation is **Moran's I** which is implemented in the **pySAL** Python module (https://pysal.org/pysal/). PySAL offers spatial analysis tools for:
- detection of spatial clusters, hot-spots, and outliers
- construction of graphs from spatial data
- spatial regression and statistical modeling on geographically embedded networks
- spatial econometrics
- exploratory spatio-temporal data analysis

The first thing we have to do when we want to calculate the spatial relationship between observations is to establish these relations in a so-called **spatial weight matrix** (**W**). A spatial weights matrix is a representation of the spatial structure of your data. It is a **quantification of the spatial relationships that exist among the features in your dataset** (or, at least, a quantification of the way you conceptualize those relationships).

Depending on the spatial process you may conceptualize spatial relationships very differently. They may reflect adjacency (e.g. A neighbors B, but not C), distance (e.g. A is 0m apart from B, and 100m from C) or complete differnt things like commuter traffic (e.g. A to B commuter traffic is 10, and A to C is 5).

Let's say we want to conceptualize and quantify the spatial relationship between South American countries and therefore create a spatial weight matrix ```W_samerica```. First, we extract all countries from ```world_map``` that are part of the "South America" continent

In [ ]:
samerica = world_map[world_map["continent"] == "South America"]
samerica = samerica[samerica["admin"] != "Falkland Islands"]
samerica.plot(figsize=(5,10))

One common approach to define the spatial relationship between polygons (areas) is to use the **Queen's scheme**. Analogous to the chessboard shown above, the neighborhood (1) or non-neighborhood (0) between areas is defined in the style of the movement of the queen piece in chess. Accordingly, areas that touch each other vertically, horizontally or diagonally are adjacent to each other. Beside's queen's and rook's there many other schemes to construct W https://pysal.org/libpysal/api.html.

We can calculate a queen style W directly from our geopandas dataframe by using ```Queen.from_dataframe()``` from pySAL's subpackage ```libpysal```.

In [ ]:
from libpysal.weights import Queen

W_samerica = Queen.from_dataframe(samerica)

The resulting object gives us access to some information, like the number ```.n``` of observations in W.

In [ ]:
W_samerica.n

This should be the same as the number of features in our dataframe.

In [ ]:
samerica.shape

Other options are the ```.max_neighbors``` of any feature in W...

In [ ]:
W_samerica.max_neighbors

...and the ```.histogram``` of the frequency of neighbours.

In [ ]:
W_samerica.histogram

Which we can plot by making this list of tuples a pandas dataframe.

In [ ]:
pd.DataFrame(W_samerica.histogram, columns=['n_neighbor', 'frequency']).plot(kind='bar', x='n_neighbor')

The values in W are actually stored in a so-called **sparse** matrix which we can access using ```.sparse```. In a sparse matrix, only non-zero elements are stored and located by their "coordinates" in the matrix. This saves *a lot* of memory, because spatial weight matrices are of size n_features * n_features.

In [ ]:
print(W_samerica.sparse)

In order to see the "normal" **dense** matrix, we can use ```.todense()``` on the sparse one. Before that, we have to make sure that numpy shows us the full matrix.

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)

print("Size of matrix:", W_samerica.sparse.todense().shape)
W_samerica.sparse.todense()

Let's use this W to calculate the spatial autocorrelation of the population density distribution in South America. We can get the area of the features using ```.area```.

In [ ]:
samerica_areas = samerica.to_crs(epsg=3857).area / 1000000
samerica_areas

Let's calcualte the number of people per square km.

In [ ]:
samerica["pop_dens"] = samerica["pop_est"] / samerica_areas

In [ ]:
samerica.plot(figsize=(5,10), column="pop_dens", legend=True)

To calculate Moran's I, we need the ```Moran()``` function from the pySAL ```esda``` sub-package and pass both the variables we are intersted in (population density) and their spatial relationships enconded in the spatial weight matrix W. 

Additionaly, we may pass the ```permutations``` parameter to determine the number of Monte Carlo style permutations that are used to determine an expected (simulated) value of Moran's I. For each permutation, the input varibale values are shuffled and randomly assinged to our geographic features while W stays constant. For each permutation, I is calculated and over all permutations we create a distribution from which we can derive an expected I for the data and spatial relationships at hand. Instead of using a simulated expected value of I, we can also use the expected value under **normality assumption**, which equals
\begin{equation*}
E(I) =   \frac{-1}{N-1}
\end{equation*}
where N is the number of features. Hereby, we assume that the observations were drawn from a normal distributed population. If we cannot assume that there is also a **randomization assumptions**.

In [ ]:
from esda.moran import Moran

morans_I = Moran(samerica["pop_dens"], W_samerica, permutations=100)

The resulting I can be derived using ```.I```.

In [ ]:
morans_I.I

The expected I under normality assumption is:

In [ ]:
morans_I.EI

The associated p-value of our I under normality assumption is:

In [ ]:
morans_I.p_norm

The associated p-value of our I under randomization assumption is:

In [ ]:
morans_I.p_rand

Therefore we observe positive spatial autocorrelation (clustering) which is statistically significant (90% confidence) under the normality assumption though. Therefore we can reject CSR under this assumption. If we follow the randomization assumption, we observe statistically significant positive autocorrelation. 

But what about our simulated expected value of I? Let's have a look a the simulated values of I from 100 permutations using ```.sim```.

In [ ]:
pd.Series(morans_I.sim).plot(kind="hist")

The expected simulated value of I ```.EI_sim``` and the associated p-value ```.p_rand``` of our observed I is: 

In [ ]:
morans_I.EI_sim

In [ ]:
morans_I.p_sim

Based on these simulations, we observe statistically significant postive spatial autocorrelation (=clustering) in our population density data.

Dive deeper into the different pySAL compontents (https://pysal.org/pysal/api.html) for stuff like:
- Exploratory Spatial Data Analysis
- Spatial regression models
- Spatial network analysis
and much more.